In [1]:
# Import libraries 
import extract_eye_features
import dill
import scipy
import os
import numpy as np
import sys 

sys.path.append("/Users/zacharykelly/Documents/MATLAB/projects/lightLogger/raspberry_pi_firmware/utility")
import Pi_util
import matplotlib.pyplot as plt
import BLNK_eye_extraction_pipeline

Loading DLC 3.0.0rc8...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [ ]:
# Define the path to the folder containing the videos and other files 
path_to_video_folder: str = "/Users/zacharykelly/Desktop/BLNK_1010"

# Generate paths to all of the videos in said folder 
video_paths: list[str] = [os.path.join(path_to_video_folder, filename)
                          for filename in os.listdir(path_to_video_folder)
                          if '.avi' in filename
                         ]

In [ ]:
# Generate output folder 
output_folder_path: str = "/Users/zacharykelly/Aguirre-Brainard Lab Dropbox/Zachary Kelly/BLNK_analysis/PuffLight/lightLevel/BLNK_1010"
if(not os.path.exists(output_folder_path)):
    os.mkdir(output_folder_path)

In [13]:
# Generate a temp output directory 
temp_dir_path: str = './temp_blnk_pipeline'
if(not os.path.exists(temp_dir_path)):
    os.mkdir(temp_dir_path)

# Define the crop box and target size 
t, b, l, r = 140, 275, 190, 425
crop_box: tuple = (t, b, l, r)
target_size: tuple = (480, 640)

# Iterate over the videos and output them to the target directory 
for video_num, video_path in enumerate(video_paths):
    print(f"Processing video: {video_num}/{len(video_paths)}", flush=True)
    BLNK_eye_extraction_pipeline.predict_eye_features(video_path, output_folder_path, crop_box, target_size, temp_dir_path)



    

Processing video: 0/26
Model weights already exist!
Analyzing videos with /Users/zacharykelly/Library/Application Support/pylids/pytorch-pupil-v1/dlc-models-pytorch/iteration-0/santini_eyelid_detectionJul182025-trainset99shuffle1/train/snapshot-best-220.pt
Starting to analyze temp_blnk_pipeline/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-001_side-R.avi
Video metadata: 
  Overall # of frames:    5760
  Duration of video [s]:  32.00
  fps:                    180.0
  resolution:             w=640, h=480

Running pose prediction with batch size 8


100%|██████████| 5760/5760 [03:16<00:00, 29.24it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpr4aboogh/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-001_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpr4aboogh/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-001_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpr4aboogh/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-001_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5']


100%|██████████| 5760/5760 [04:20<00:00, 22.10it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp6vysdz0e/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-001_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp6vysdz0e/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-001_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp6vysdz0e/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-001_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110.h5']


...

Processing video: 25/26
Model weights already exist!
Analyzing videos with /Users/zacharykelly/Library/Application Support/pylids/pytorch-pupil-v1/dlc-models-pytorch/iteration-0/santini_eyelid_detectionJul182025-trainset99shuffle1/train/snapshot-best-220.pt
Starting to analyze temp_blnk_pipeline/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.30_trial-007_side-R.avi
Video metadata: 
  Overall # of frames:    5760
  Duration of video [s]:  32.00
  fps:                    180.0
  resolution:             w=640, h=480

Running pose prediction with batch size 8


100%|██████████| 5760/5760 [03:28<00:00, 27.60it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpqr2nfkim/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.30_trial-007_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpqr2nfkim/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.30_trial-007_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpqr2nfkim/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.30_trial-007_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5']


100%|██████████| 5760/5760 [05:09<00:00, 18.59it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpeypwisqq/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.30_trial-007_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpeypwisqq/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.30_trial-007_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpeypwisqq/temp_BLNK_1009_lightLevel_direction-LightFlux_sequence-1_contrast-0.30_trial-007_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110.h5']
